In [ ]:
import csv
import os
from tabulate import tabulate

class Product:
    def __init__(self, name, number, purchase_price, selling_price):
        self.name = name
        self.number = number
        self.purchase_price = purchase_price
        self.selling_price = selling_price

store = []
gross_profits_list = []
net_profits_list = []

def upload_data():
    global store
    try:
        if os.path.exists('store.csv'):
            with open('store.csv', mode='r') as file:
                reader = csv.reader(file)
                for row in reader:
                    if len(row) == 4:
                        product = Product(row[0], int(row[1]), float(row[2]), float(row[3]))
                        store.append(product)                    
    except FileNotFoundError:
        print("Nessun file di dati trovato. Magazzino vuoto.")

def save_data():
    with open('store.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        for product in store:
            writer.writerow([product.name, product.number, product.purchase_price, product.selling_price])

def ask_number():
    while True:
        number = input("\nQuantità: ")
        if number.isdigit():
            number = int(number)
            if number >= 0:
                return number
            else:
                print("Inserisci una quantità accettabile (numero non negativo)!")
                continue
        else:
            print("Inserisci una quantità accettabile che sia un numero!")
            continue

def add_product():
    name = input("\nNome del prodotto: ")
    boolean = False 
    for product in store:
        if product.name == name:
            boolean = True
            print(f"\nProdotto {product.name} già presente in magazzino\n")
            break

    number = ask_number()
    
    if boolean:
        product.number += number
    else:
        purchase_price = float(input("Prezzo di acquisto: "))
        selling_price = float(input("Prezzo di vendita: "))
        product = Product(name, number, purchase_price, selling_price)
        store.append(product)

    if number > 0:
        print(f"AGGIUNTO: {product.number} X {name}")

def get_products():
    print("\n")
    l = [[product.name, product.number, f"€{product.selling_price}"] for product in store]
    header = ["PRODOTTO", "QUANTITA'", "PREZZO"]
    print(tabulate(l, headers=header, tablefmt="plain"))

def add_sale():
    global gross_profits_list, net_profits_list 
    products_list = []
    number_list = []
    gross_profits = 0
    net_profits = 0
    while True:
        name = input("\nNome del prodotto: ")
        boolean = False
        for product in store:
            if product.name == name:
                boolean = True
                number = ask_number()
                    
                if product.number >= number:
                    number_list.append(number)
                    product.number -= number
                    products_list.append(product)
                    
                    if product.number == 0:
                        store.remove(product)
                else:
                    print(f"Quantità non disponibile per {name}")
        if not boolean: 
            print("\nProdotto non presente in store")
            
        another_product = input("\nAggiungere un altro prodotto ? (si/no): ")

        while another_product not in ["si", "no"]:
            print("Comando inserito non valido! Digitare si o no.")
            another_product = input("\nAggiungere un altro prodotto ? (si/no): ")

        if another_product.lower() == "no":
            break

    
    if boolean:
        print("\nVENDITA REGISTRATA")
        for i, product in enumerate(products_list):
            print(f" - {number_list[i]} X {product.name}: {product.selling_price}")
            gross_profits += number_list[i] * product.selling_price
            net_profits += number_list[i] * (product.selling_price - product.purchase_price)
        print(f"Totale: €{gross_profits:.2f}")
        gross_profits_list.append(gross_profits)
        net_profits_list.append(net_profits)

def show_profits():
    global gross_profits_list, net_profits_list
    gross_profits = 0
    net_profits = 0
    
    for item in gross_profits_list:
        gross_profits += item

    for item in net_profits_list:
        net_profits += item
    
    print(f"Profitto: lordo=€{gross_profits:.2f} netto=€{net_profits:.2f}")

def help_command():
    print("\nI comandi disponibili sono i seguenti:\n\n"
          " - aggiungi: aggiungi un prodotto al magazzino\n"
          " - elenca: elenca i prodotti in magazzino\n"
          " - vendita: registra una vendita effettuata\n"
          " - profitti: mostra i profitti totali\n"
          " - help_comand: mostra i possibili comandi\n"
          " - chiudi: esci dal programma\n")

commands = {
    "aggiungi": add_product,
    "elenca": get_products,
    "vendita": add_sale,
    "profitti": show_profits,
    "help_comand": help_command,
}

if __name__ == "__main__":
    upload_data()
    
    running = True
    while running:
        command = input("\nInserisci un comando: ")
        if command in commands:
            commands[command]()
        elif command == "chiudi":
            running = False
            save_data()
            print("\nBye Bye")
        else:
            print("\nComando non valido")
            help_command()

